In [1]:
!apt-get -qq install postgresql postgresql-contrib
!pip install psycopg2-binary

# ---------- START POSTGRES ----------
!service postgresql start

import psycopg2

# ---------- CREATE USER AND DB ----------
!sudo -u postgres psql -c "CREATE USER advisor WITH PASSWORD 'advisor123';"
!sudo -u postgres psql -c "ALTER USER advisor CREATEDB;"
!sudo -u postgres psql -c "CREATE DATABASE advisor_db OWNER advisor;"

# ---------- CONNECT TO POSTGRES ----------
conn = psycopg2.connect(
    dbname="advisor_db",
    user="advisor",
    password="advisor123",
    host="localhost",
    port=5432
)
conn.autocommit = True
cursor = conn.cursor()

# ---------- CREATE TABLE ----------
cursor.execute("""
DROP TABLE IF EXISTS samsung_phones;
CREATE TABLE samsung_phones (
    model TEXT,
    display TEXT,
    battery INT,
    camera TEXT,
    ram TEXT,
    storage TEXT,
    price INT
);
""")

# ---------- INSERT SAMPLE DATA ----------
cursor.executemany("""
INSERT INTO samsung_phones VALUES (%s,%s,%s,%s,%s,%s,%s)
""", [
    ("Galaxy S23 Ultra", "6.8 AMOLED", 5000, "200MP", "12GB", "256GB", 1199),
    ("Galaxy S22 Ultra", "6.8 AMOLED", 5000, "108MP", "12GB", "256GB", 999),
    ("Galaxy S23", "6.1 AMOLED", 3900, "50MP", "8GB", "128GB", 799),
    ("Galaxy A54", "6.4 AMOLED", 5000, "50MP", "8GB", "128GB", 449)
])

# =========================================================
# RAG MODULE: Extract relevant phone data
# =========================================================
def data_extractor(question):
    cursor.execute("SELECT * FROM samsung_phones;")
    rows = cursor.fetchall()
    return [r for r in rows if r[0].lower() in question.lower()]

# =========================================================
# Multi-Agent: Review / Recommendation Generator
# =========================================================
def review_generator(data, question):
    q = question.lower()

    if "compare" in q and len(data) >= 2:
        a, b = data[:2]
        return (
            f"{a[0]} has a {a[3]} camera and {a[2]}mAh battery, "
            f"while {b[0]} has a {b[3]} camera and {b[2]}mAh battery. "
            f"{a[0]} is recommended for photography."
        )

    if "best battery" in q:
        best = max(data, key=lambda x: x[2])
        return f"{best[0]} offers the best battery life ({best[2]}mAh)."

    phone = data[0]
    return (
        f"{phone[0]} has a {phone[1]} display, "
        f"{phone[2]}mAh battery, {phone[3]} camera, "
        f"{phone[4]} RAM and {phone[5]} storage."
    )

# =========================================================
# Unified “ask” function (simulate endpoint)
# =========================================================
def ask(question):
    data = data_extractor(question)
    if not data:
        return {"answer": "No matching Samsung phone found."}
    return {"answer": review_generator(data, question)}

# =========================================================
# TEST QUESTIONS
# =========================================================

print(ask("Compare Galaxy S23 Ultra and Galaxy S22 Ultra"))
print(ask("Which Samsung phone has the best battery under $1000?"))
print(ask("What are the specs of Galaxy A54?"))

 * Starting PostgreSQL 14 database server
   ...done.
ERROR:  role "advisor" already exists
ALTER ROLE
ERROR:  database "advisor_db" already exists
{'answer': 'Galaxy S23 Ultra has a 200MP camera and 5000mAh battery, while Galaxy S22 Ultra has a 108MP camera and 5000mAh battery. Galaxy S23 Ultra is recommended for photography.'}
{'answer': 'No matching Samsung phone found.'}
{'answer': 'Galaxy A54 has a 6.4 AMOLED display, 5000mAh battery, 50MP camera, 8GB RAM and 128GB storage.'}
